In [5]:
import pyodbc
import pandas as pd
import numpy as np

In [6]:
# Database connection
server = 'server_name'
db = 'db_name'
user = 'user'
password = 'pass'

In [7]:
# To check for problems with the connection_manager and which driver to use.
"""
# List of installed ODBC drivers
drivers = [driver for driver in pyodbc.drivers()]]

# Print available ODBC drivers
print("ODBC drivers installed:")
for driver in drivers:
    print(driver)

"""

'\n# Lista de controladores ODBC instalados\ndrivers = [driver for driver in pyodbc.drivers()]\n\n# Imprimir los controladores ODBC disponibles\nprint("Controladores ODBC instalados:")\nfor driver in drivers:\n    print(driver)\n\n'

In [8]:
# Conection manager
try:
    #connection_string = 'DRIVER={SQL Server Native Client 11.0};SERVER='+server+';DATABASE='+db+';UID='+user+';PWD='+password+''
    connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={db};UID={user};PWD={password}'
    print('successful connection')

except pyodbc.Error as e:
    print('Error al conectar a la base de datos:', e)



successful connection


In [9]:
# Connect to the database
conn = pyodbc.connect(connection_string)

# Create a cursor
cursor = conn.cursor()

# Define the SQL query
query = "SELECT [bst_nr_8], [prod_datum], [number_trips], [minutes] FROM [DWH].[dbo].[deutschebahn_real_trafic_data] WITH (NOLOCK) WHERE [country] = 'DEUTSCHLAND'"

# Execute the query
cursor.execute(query)

# Get the results
resultados = cursor.fetchall()

# Execute the query with the value
df = pd.read_sql(query, conn)

# Close conection
cursor.close()
conn.close()


C:\Users\alexm\AppData\Local\Temp\ipykernel_11920\621900380.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [10]:
df

,bst_nr_8,prod_datum,number_trips,minutes
0,80197079,2016-03-16,39,49
1,80160168,2016-11-01,12,0
2,80105122,2016-04-30,16,2
3,80190793,2016-07-31,14,0
4,80231464,2016-05-04,10,2
...,...,...,...,...
771616,80136234,2016-10-26,82,0
771617,80100388,2016-01-29,33,2
771618,80143321,2016-05-30,40,0
771619,80174508,2016-07-05,14,0


In [11]:
# Filter data by country
df_filtered = df

# Convert column prod_datum to type DATE
df_filtered['prod_datum'] = pd.to_datetime(df_filtered['prod_datum']).dt.date

# Calculate avg_delay_per_trip column
df_filtered['avg_delay_per_trip'] = df_filtered['minutes'] / df_filtered['number_trips']
df_filtered['avg_delay_per_trip'] = df_filtered['avg_delay_per_trip'].round(2)

# Select the required columns (just to be more specific)
columns_to_select = ['bst_nr_8', 'prod_datum', 'number_trips', 'minutes', 'avg_delay_per_trip']
result_df = df_filtered[columns_to_select]

# Sort dataframe
result_df = result_df.sort_values(by=['prod_datum', 'number_trips', 'minutes'], ascending=[False, False, False])

# Reset the DataFrame index (like the original df)
result_df = result_df.reset_index(drop=True)

print(result_df)


        bst_nr_8  prod_datum  number_trips  minutes  avg_delay_per_trip
0       80011346  2016-11-30           245      308                1.26
1       80011833  2016-11-30           241      230                0.95
2       80135079  2016-11-30           207      246                1.19
3       80135095  2016-11-30           197      203                1.03
4       80135624  2016-11-30           193     1148                5.95
...          ...         ...           ...      ...                 ...
771616  80113811  2016-01-15            10        0                0.00
771617  80195867  2016-01-15            10        0                0.00
771618  80163154  2016-01-15            10        0                0.00
771619  80295220  2016-01-15            10        0                0.00
771620  80295816  2016-01-15            10        0                0.00

[771621 rows x 5 columns]


In [16]:
# Total
total = len(result_df)

# With delay
delay = len(result_df[result_df['minutes'] > 0])

# Without delay
correct = len(result_df[result_df['minutes'] <= 0])

# Percentage delay
percentage_delay = (delay / total) * 100

# Percentage without delay
percentage_correct = (correct / total) * 100

In [15]:
print(f'In the year 2016 there were a total of {total}  trips in Germany, of which {percentage_delay:.2f}% had a delay of at least 1 minute \n while those with no delay were a total of {percentage_correct:.2f}%')


In the year 2016 there were a total of 771621  trips in Germany, of which 59.58% had a delay of at least 1 minute 
 while those with no delay were a total of 40.42%
